# Initialize

## ASW session

In [98]:
import boto3

session = boto3.Session()
s3 = session.resource('s3')

## SageMaker Session

In [99]:
import sagemaker

sage_session = sagemaker.Session()

# Define Globals

In [109]:
# bucket name
bucket = 'aida-project'

# path for s3_input
s3_path = f"s3://{bucket}/"

## define source files
# TRAIN source file
train_source_path = 'team_remote'
train_source_name = 'title-train-princ4.csv'

# VALID source file
valid_source_path = 'team_remote'
valid_source_name = 'title-valid-princ4.csv'

# TEST source file
test_source_path = 'team_remote'
test_source_name = 'title-test.csv'

# output for result
output_path ='team_remote'
output_location = f'{s3_path}{output_path}output'

In [110]:
# Create pointers to the S3 train and test datasets

from sagemaker import s3_input

# print(f"{s3_path}{train_source_path}/{train_source_name}")
# print(f"{s3_path}{valid_source_path}/{valid_source_name}")
# print(f"{s3_path}{test_source_path}/{test_source_name}")
s3_input_train = sagemaker.session.s3_input(s3_data=f"{s3_path}{train_source_path}/{train_source_name}", content_type="text/csv")
s3_input_valid = sagemaker.session.s3_input(s3_data=f"{s3_path}{valid_source_path}/{valid_source_name}", content_type="text/csv")
s3_input_test = sagemaker.session.s3_input(s3_data=f"{s3_path}{test_source_path}/{test_source_name}", content_type="text/csv")

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


In [102]:
role = sagemaker.get_execution_role()

# Define Model

### Factorization Machine

In [103]:
# Factorization Machine

# estimator call
# <ecr_path>/factorization-machines:<tag>

ecr_path = '664544806723.dkr.ecr.eu-central-1.amazonaws.com'
tag = 'latest'
estimator = f'{ecr_path}/factorization-machines:{tag}'


In [104]:
JPC_factor = sagemaker.estimator.Estimator(estimator,
                              role,
                              train_instance_count=1,
                              train_instance_type="ml.m5.4xlarge",
                              output_path=output_location,
                              sagemaker_session=sage_session,
                              base_job_name="JPC-factor")

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [116]:
JPC_factor.set_hyperparameters(
    feature_dim = 74, # Anzahl der Spalten in den Daten + 1. y
    num_factors = 64, # auf wieviele soll er reduzieren - wonach er die Ähnlichkeit feststellt (# je mehr, desto genauer - aber bei vielen = overfitting )
    predictor_type = 'regressor'
    )

In [112]:
feature_importance_col_names = ['isAdult', 'startYear', 'endYear', 'numVotes', 'Crime', 'Romance', 'Thriller', 'Adventure', 'Drama', 'War', 'Documentary', 'Reality-TV', 'Family', 'Fantasy', 'Game-Show', 'Adult', 'History', 'Mystery', 'Musical', 'Animation', 'Music', 'Film-Noir', 'shortGenre', 'Horror', 'Western', 'Biography', 'Comedy', 'Action', 'Sport', 'Talk-Show', 'Sci-Fi', 'News', 'movie', 'short', 'tvEpisode', 'tvMiniSeries', 'tvMovie', 'tvSeries', 'tvShort', 'tvSpecial', 'video', 'videoGame', 'nm0592314', 'nm10514069', 'nm11131030', 'nm11131593', 'nm1915152', 'nm2124199', 'nm2299581', 'nm2699546', 'nm3816718', 'nm4837590', 'nm5181169', 'nm5511980', 'nm6135542', 'nm6419589', 'nm7155068', 'nm7702567', 'nm8062068', 'nm8079243', 'nm8269207', 'nm8409754', 'nm8641187', 'nm8688464', 'nm8713978', 'nm8741892', 'nm9002892', 'nm9099684', 'nm9099685', 'nm9177141', 'nm9325202', 'nm9354107', 'distinct languages', 'distinct regions']
len(feature_importance_col_names)

74

In [117]:
# es scheint nicht möglich zu sein nur auf dem validationset das training zu machen: fehler: ihm fehlt das validationset

JPC_factor.fit({"train": s3_input_valid, "validation": s3_input_valid}, wait=True)

2020-11-26 15:46:05 Starting - Starting the training job...
2020-11-26 15:46:07 Starting - Launching requested ML instances......
2020-11-26 15:47:12 Starting - Preparing the instances for training...
2020-11-26 15:47:56 Downloading - Downloading input data...
2020-11-26 15:48:25 Training - Downloading the training image.Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python2.7/site-packages/pandas/util/nosetester.py:13: DeprecationWarning: Importing from numpy.testing.nosetester is deprecated, import from numpy.testing instead.
  from numpy.testing import nosetester
[11/26/2020 15:48:42 INFO 140186482612032] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'factors_lr': u'0.0001', u'linear_init_sigma': u'0.01', u'epochs': 1, u'_wd': u'1.0', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'factors_init_sigma': u'0.001', u'_log_level': u'info', u'bias_

UnexpectedStatusException: Error for Training job JPC-factor-2020-11-26-15-46-05-452: Failed. Reason: ClientError: Found at least 2 entries in file /opt/ml/input/data/train/title-valid-princ4.csv with missing values:
Row 8232, column 3
Row 8232, column 4

### zweiter Ansatz: skip Row when loading Training Set:

In [96]:
# der Path ist nicht richtig = welchen Path könnte ich noch angeben?

import pandas as pd

df1 = pd.read_csv(s3_input_train, skiprows = 1)
df1.head()

ValueError: Invalid file path or buffer object type: <class 'sagemaker.inputs.s3_input'>

In [ ]:
JPC_factor.deploy(
#     initial_instance_count = 1, 
#     instance_type = "ml.m5.4xlarge")

### neu Matrix Factorization

In [ ]:
! python

Python 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 

In [ ]:
! pip install pymf3

In [ ]:
import cvxopt, numpy, and scipy

### ALS Model: 
https://towardsdatascience.com/prototyping-a-recommender-system-step-by-step-part-2-alternating-least-square-als-matrix-4a76c58714a1

In [52]:
def _jvm():
    """
    Returns the JVM view associated with SparkContext. Must be called
    after SparkContext is initialized.
    """
    jvm = SparkContext._jvm
    if jvm:
        return jvm
    else:
        raise AttributeError("Cannot load _jvm from SparkContext. Is SparkContext initialized?")

In [50]:
from pyspark.ml.recommendation import ALS
from pyspark import SparkContext


def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
    """
    grid search function to select the best model based on RMSE of
    validation data
    Parameters
    ----------
    train_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    validation_data: spark DF with columns ['userId', 'movieId', 'rating']
    
    maxIter: int, max number of learning iterations
    
    regParams: list of float, one dimension of hyper-param tuning grid
    
    ranks: list of float, one dimension of hyper-param tuning grid
    
    Return
    ------
    The best fitted ALS model with lowest RMSE score on validation data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            als = ALS().setMaxIter(maxIter).setRank(rank).setRegParam(reg)
            # train ALS model
            model = als.fit(train_data)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(validation_data)
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="rating",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {}'.format(best_rank, best_regularization))
    return best_model

In [54]:
tune_ALS(train_source_name, valid_source_name, 2, [0.1,0.2], [10.0,20.0])

AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?

### XGBoost

In [53]:
# find pre-defined models here:
# https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html

# Create an XGBoost Estimator

# estimator call
# <ecr_path>/sagemaker-xgboost:1.2-1
    
ecr_path = "492215442770.dkr.ecr.eu-central-1.amazonaws.com"
estimator = f"{ecr_path}/sagemaker-xgboost:1.0-1-cpu-py3"


In [15]:
xgboost = sagemaker.estimator.Estimator(estimator,
                              role,
                              train_instance_count=1,
                              train_instance_type="ml.m5.4xlarge",
                              output_path=output_location,
                              sagemaker_session=sage_session,
                              base_job_name="JPC-xgboost")

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [16]:
# Select the your specific hyperparameters (Optional)
# https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html
# https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst#learning-task-parameters 

xgboost.set_hyperparameters(
    eta=.35,
    num_round=30,                     # required parameter!
    objective = 'reg:logistic',
    )

## Fit Model

In [ ]:
#xgboost.fit({"train": s3_input_train, "validation": s3_input_valid}, wait=True)

# Deploy Model

## Deploy

In [ ]:
# Deploy  model to an endpoint

# xgb_predictor = xgboost.deploy(
#     initial_instance_count = 1, 
#     instance_type = 'ml.t2.medium')

## Configure Predictor

In [ ]:
# Configure the predictor's serializer and deserializer

# INSERT CODE HERE

from sagemaker.predictor import csv_serializer, json_deserializer

# xgb_predictor.content_type = "text/csv"
# xgb_predictor.serializer = csv_serializer
# xgb_predictor.deserializer = json_deserializer

# Display Endpoint

In [18]:
# xgb_predictor.endpoint

# Prediction

In [20]:
# carefully just one prediction
# prediction = xgb_predictor.predict(test.iloc[0, 1:])
# prediction